# Assignment 3

## Exercise
Documentation and code of CLIP: https://github.com/openai/CLIP

In this notebook, you will be working with CLIP (Contrastive Language-Image Pre-Training) model. We will be using the implementation and the pre-trained model based on ViT. This is a visual transformer, see the paper for details, or the (upcoming) course slides on ILIAS.

In [1]:
import os
import sys

import numpy as np
import torch
print("Torch version:", torch.__version__)

try:
    import pandas
except ModuleNotFoundError:
    !conda install pandas

Torch version: 2.0.1


In [ ]:
#conda update -n base -c defaults conda
#conda install torchvision -c pytorch

In [2]:
WORKWITHCOLAB = False

# Setup: Preparation for Colab
***If you work on your own local environment, you can skip this part and go directly to the Warm-Up.***

Make sure you're running a GPU runtime; if not, select "GPU" as the hardware accelerator in Runtime > Change Runtime Type in the menu. The next cells will install the `clip` package and its dependencies, and check if PyTorch 1.7.1 or later is installed.

In [3]:
if WORKWITHCOLAB:
    !pip install cython

In [4]:
if WORKWITHCOLAB:
    try:
        from google.colab import drive
    except ModuleNotFoundError:
        !pip install google-colab

In [5]:
# Mount your Google Drive to the Colab VM:
if WORKWITHCOLAB:
    from google.colab import drive
    drive.mount('/content/drive')

    # TODO: Save the unzipped assignment folder in a filder on your google Drive. 
    # TODO: Enter the foldername in your Drive where you have saved it, e.g. 'assignment4/'
    FOLDERNAME = None
    FOLDERNAME = "MultimodalSemantics_SS2023"
    assert FOLDERNAME is not None, "enter the foldername."

    # Now that we've mounted your Drive, this ensures that
    # the Python interpreter of the Colab VM can load
    # python files from within it.
    sys.path.append('/content/drive/MyDrive/{}'.format(FOLDERNAME))

In [6]:
if WORKWITHCOLAB:
    %load_ext Cython

In [7]:
if WORKWITHCOLAB:
    !cd /content/drive/MyDrive/$FOLDERNAME/
    !ls .

# Setting up working with CLIP

This is a self-contained notebook that shows how to download and run CLIP models, calculate the similarity between arbitrary image and text inputs, and perform zero-shot image classifications.

In [8]:
# Install CLIP if you don't have it yet
! pip install ftfy regex tqdm
! pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-_kmlo3e0
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-_kmlo3e0
  Resolved https://github.com/openai/CLIP.git to commit a9b1bf5920416aaeaec965c25dd9e8f98c864f16
  Preparing metadata (setup.py) ... done


In [9]:
from pkg_resources import packaging

In [10]:
import torchvision
print("Torch vision version: ", torchvision.__version__)
#import torch
#torch.__version__

Torch vision version:  0.15.2


In [ ]:
# If necessary:
#!conda update --yes pytorch torchvision

In [ ]:
# Getting a warn(f"Failed to load image Python extension: {e}") error?
# Fix:
#!pip3 uninstall --yes torchvision
#!pip3 install torchvision==0.12.0

In [11]:
# We need additional packages for loading (and visualising) the images
import IPython.display
import matplotlib.pyplot as plt
from collections import OrderedDict

try:
    import skimage
except ModuleNotFoundError:
    !conda install --yes scikit-image 
    import skimage
from PIL import Image

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Loading the model and the image preprocessor

`clip.available_models()` will list the names of available CLIP models.

In [12]:
import clip
clip.available_models()

['RN50',
 'RN101',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'ViT-B/32',
 'ViT-B/16',
 'ViT-L/14',
 'ViT-L/14@336px']

We load the pre-trained ViT-B/32 `model` and the image preprocessor (`process`)

In [13]:
model, preprocess = clip.load("ViT-L/14")
if torch.cuda.is_available():
    model.cuda().eval()
else:
    model.eval()
input_resolution = model.visual.input_resolution
context_length = model.context_length
vocab_size = model.vocab_size

print("Model parameters:", f"{np.sum([int(np.prod(p.shape)) for p in model.parameters()]):,}")
print("Input resolution:", input_resolution)
print("Context length:", context_length)
print("Vocab size:", vocab_size)

Model parameters: 427,616,513
Input resolution: 224
Context length: 77
Vocab size: 49408


## Exercise: Language Perception of Colour II
In the exercises, you will use the coda dataset as in the previous assignment: https://github.com/nala-cub/coda

This exercise now evaluates VL-models on the colour prediction task. 
So, again, follow the instructions below. 

### Instructions
* Load the coda dataset. We will only use the *object names* and *colours* for this exercise.
* Take the same 20 object names from different categories (e.g., vegetable/fruits, vehicles, etc.) that you used in Assignment 2.
* Use the reference data of colour attributions that you created in Assigment 2.
* The task now is to use CLIP for predicting the top colours that for each of your objects, as explained below. 
 
#### Approach
We will compare different methods that diverge in the information that the modalities capture, and will perform zero-shot classification to predict typical colours of objects using CLIP. <br/>
Regarding the prompts mentioned below, see also the file `templates.yaml` of the coda dataset.

##### Data Preparation
You need to set up your coda data and the data structures to extract the information needed for the methods. See the setup of the `swig_data` conducted above for zero-shot classification. 

1. **CLIP-col**: 
   *  Visual embeddings of images of colours   <br/>
      Retrieve images from the web of all target colours under study, and store them in `coloured_objects`. Do not forget to give the source of the images in your report.
    * For the textual representations, adapt the prompts you used in the previous assignment, but use the phrases "this colour/these colours" instead of masking the colour words. 
      * For example, you could use the template 
     `"Most {object_areis_pl} have this colour."` <br/>
     
2. **CLIP-Tcol**: Baseline to **CLIP-col** without using images:
   * "Visual embeddings" are actually textual embeddings of the colour *words*   <br/>
      That is, use the textual encoder to extract the "visual embeddings" of each colour term.  
    * For the textual representations, follow CLIP-col.<br/>

3. **CLIP-obj**: 
   Here, you first have to define your target classes. These will be the target colours. 
   * Visual embeddings of images of the target objects <br/>
      These images are grey-scale drawings. You can retrieve corresponding images from the MultPIC database, or query others (report the source of the photos). 
   * For the textual representations use simple prompts, such as <br/>
       `"This object usually is {}."`, <br/>
       `"This object usually is {} coloured."`

### Tasks 1: Analysis of the VL-Models
1. Briefly give an overview (in a table) of the methods in terms of with which modality they encode the *target object* and the *target colour(s)*, respectively. 
  | Method    | Image | Language |
  | ------    | ----- | -------- |
  | CLIP-col  |   ?   |    ?     |
  | CLIP-Tcol |   ?   |    ?     |
  | CLIP-obj  |   ?   |    ?     |
2. Compute the accuracy of each model on the colour prediction task using your small reference data (previous coda assignment).
3. Compare the effectiveness of the models, and discuss the possible reasons for the differences. 
4. For each model, are there objects for which it predicts wrong colours? Why could this be the case?
5. Summarise in a few sentences your conclusions regarding the colour knowledge one can learn from visual-linguistic representations (i.e., from the visual modality and from text use). Also take into account the different models. 

### Tasks 2: Comparison of Language Model  against VL-Model
*See the pdf of the assignment.*

### Task 3: Pen & Paper Questions
*See the pdf of the assignment.*

***Submit the following files through ILIAS***:
* `03ex_vlmodels_clip.ipynb`: your completed jupyter notebook
* `03ex_vlmodels_clip.[txt|md]`: a plain text file that contains your answers to the task and questions.


### Setting up coloured_objects<a name="setup_coloured_objects" />
#### EXAMPLE:

In [14]:
# TODO: set up your paths.
FOLDERNAME = "data"
image_dir = '{}/coloured_objects/'.format(FOLDERNAME)
data_dir = '{}/coloured_objects/'.format(FOLDERNAME)

for filename in [filename for filename in os.listdir(data_dir)]:
    print(filename)

banana_gratismalvorlagen.png
banana_wikipedia.jpg
peach_supercoloring.png
peeling_91.jpg
PICTURE_42.png
PICTURE_69.png
PICTURE_693.png
PICTURE_697.png
PICTURE_707.png


In [ ]:
# # images in coda to use and some descriptions for them written by yourselves
# descriptions = [
#     ("PICTURE_69.png", "A grey-scale photo of a banana ."),
#     ("PICTURE_42.png", "A photo of a pear ."),
# ]

In [ ]:
# colour_classes = ['yellow', 'blue', 'brown', 'pink', 'gray', 'black', 'red', 'green', 'orange', 'purple', 'white']
# colour_class2idx = {c:idx for (idx,c) in enumerate(colour_classes)}

In [ ]:
# coloured_objects_data = {
#     # Q: How to adapt to multi-label prediction?
#     "PICTURE_69.png": "yellow",
#     "PICTURE_42.png": "green",
# }

In [ ]:
# # For zero-shot classification
# # We want to use more than one prompt, and get a 
# # mean prompt embedding for them: 
# colour_templates = ["A photo of an object that is usually {} coloured .", 
#                    "The color of this object on the photo usually is {} ."]
# print(f"{len(colour_classes)} classes, {len(colour_templates)} templates")

In [20]:
## YOUR CODE COMES HERE

# load coda
from datasets import load_dataset

coda_dataset = load_dataset("corypaik/coda")
coda_dataset = coda_dataset["train"]

Found cached dataset coda (/home/kafka/.cache/huggingface/datasets/corypaik___coda/default/1.0.1/cebd02670e856cb431040f8050cc5e0dcf887096aa9a62857be5596abfe23656)


  0%|          | 0/3 [00:00<?, ?it/s]

In [23]:
coda_dataset.features

{'class_id': Value(dtype='string', id=None),
 'display_name': Value(dtype='string', id=None),
 'ngram': Value(dtype='string', id=None),
 'label': Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None),
 'object_group': ClassLabel(names=['Single', 'Multi', 'Any'], id=None),
 'text': Value(dtype='string', id=None),
 'template_group': ClassLabel(names=['clip-imagenet', 'text-masked'], id=None),
 'template_idx': Value(dtype='int32', id=None)}

In [27]:
# randomly select 20
# selected_indexes = np.random.choice(len(object_names), 20)
# selected_objects = [object_names[idx] for idx in selected_indexes]
# print(selected_objects)

selected_objects = ['Ambulance', 'Crow', 'Ketchup', 'Mobile phone', 'Porcupine', 
                    'Apple', 'Microphone', 'Skyscraper', 'Limousine',
                    'Canoe', 'Light bulb', 'Asparagus', 'Violin', 'Sea turtle', 
                    'Skull', 'Giraffe', 'Traffic Light', 'Drawer', 
                    'Jeans', 'Flashlight']

assert len(selected_objects) == 20

In [28]:
reference_colors = ["white", "black", "red", "black", "brown", "red", "black", 
                    "gray", "white", "white", "yellow", "green", "black", 
                    "green", "white", "yellow", "red", "white", "blue", "white"]

assert len(selected_objects) == len(reference_colors)